In [1]:
import src.models.builders as mb
import src.models.helpers as mh
import src.inference.helpers as ih
import plotly.graph_objects as go
import src.modelling_o2.fit_ho2sat_ar as fit_ho2sat_ar
import src.models.o2_fev1_point_in_time as model

import numpy as np

In [2]:
height = 175
age = 30
sex = "Male"
hfev1_prior = {"type": "default", "height": height, "age": age, "sex": sex}
ho2sat_prior = {
    "type": "default",
    "height": height,
    "sex": sex,
}
(
    model,
    inf_alg,
    HFEV1,
    ecFEV1,
    AR,
    HO2Sat,
    O2SatFFA,
) = model.build(hfev1_prior, ho2sat_prior)

Defining gaussian prior with mu = 97.20, sigma = 1.0304


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:312: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(


In [19]:
HFEV1.bins_str[66]

'[4.3, 4.35)'

In [66]:
ecFEV1.bins_str[86]

'[4.3, 4.35)'

In [67]:
prior = ecFEV1.prior.reshape(120, len(HFEV1.bins), len(AR.bins))
prior.shape

(120, 100, 45)

In [81]:
print(prior[86, 66, :])

[0.28846368 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]


In [83]:
print(prior[86, 67, :])

[0.55472169 0.0167131  0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]


In [64]:
res = ih.infer(inf_alg, [HFEV1], [[ecFEV1, 4.3]], True)
res.values

  0%|          | 0/1 [00:00<?, ?it/s]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.0420966 , 0.08314325, 0.08120231, 0.07857753,
       0.07533154, 0.07154249, 0.06730087, 0.06270578, 0.05786

In [119]:
fig = go.Figure()
fig.add_trace(go.Bar(y=HFEV1.prior[:, 0], x=HFEV1.bins))
# fig.add_trace(go.Histogram(x=np.repeat(bins, p), histnorm='probability', nbinsx=len(HFEV1.bins)))
# fig.add_trace(go.Histogram(x=np.repeat(HFEV1.bins, (HFEV1.prior[:,0]*10000).astype(int)), histnorm='probability', xbins=dict(
#         start=0,
#         end=6,
#         size=HFEV1.bin_width
#     )))
fig.update_layout(bargap=0.01)
fig.update_xaxes(range=[0, 6])
fig.update_traces(marker_line_width=0)
fig.show()

In [147]:
fig = go.Figure()
# 0.04 * 0.001 = 0.00004 - include probabilities 1000 times smaller than the max probability
# fig.add_trace(go.Histogram(x=np.repeat(bins, p), histnorm='probability', nbinsx=len(HFEV1.bins)))
# fig.add_trace(go.Histogram(x=np.repeat(HFEV1.bins, (HFEV1.prior[:,0]*1000).astype(int)), histnorm='probability', xbins=dict(
#         start=0,
#         end=6,
#         size=HFEV1.bin_width
#     )))
fig.add_trace(
    go.Histogram(
        x=HFEV1.bins,
        y=HFEV1.prior[:, 0],
        histfunc="sum",  # Use 'sum' to represent pre-counted data
        xbins=dict(start=0, end=6, size=HFEV1.bin_width),
    )
)
# fig.update_layout(bargap=0.01)
fig.update_xaxes(range=[0, 6], title="FEV1 (L)")
fig.show()

In [142]:
HFEV1.bins

array([1.  , 1.05, 1.1 , 1.15, 1.2 , 1.25, 1.3 , 1.35, 1.4 , 1.45, 1.5 ,
       1.55, 1.6 , 1.65, 1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  , 2.05,
       2.1 , 2.15, 2.2 , 2.25, 2.3 , 2.35, 2.4 , 2.45, 2.5 , 2.55, 2.6 ,
       2.65, 2.7 , 2.75, 2.8 , 2.85, 2.9 , 2.95, 3.  , 3.05, 3.1 , 3.15,
       3.2 , 3.25, 3.3 , 3.35, 3.4 , 3.45, 3.5 , 3.55, 3.6 , 3.65, 3.7 ,
       3.75, 3.8 , 3.85, 3.9 , 3.95, 4.  , 4.05, 4.1 , 4.15, 4.2 , 4.25,
       4.3 , 4.35, 4.4 , 4.45, 4.5 , 4.55, 4.6 , 4.65, 4.7 , 4.75, 4.8 ,
       4.85, 4.9 , 4.95, 5.  , 5.05, 5.1 , 5.15, 5.2 , 5.25, 5.3 , 5.35,
       5.4 , 5.45, 5.5 , 5.55, 5.6 , 5.65, 5.7 , 5.75, 5.8 , 5.85, 5.9 ,
       5.95])

In [68]:
cpt_ecfev1 = mh.calc_pgmpy_cpt_X_x_1_minus_Y(HFEV1, AR, ecFEV1)

/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:312: IntegrationWarning:

The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.



KeyboardInterrupt: 

In [7]:
viz_layout = [
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
]

In [9]:
np.shape(viz_layout)

(9, 6)